# Imports

## Import libraries

In [4]:
%pip install datasets transformers evaluate
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding
import pandas as pd
import os
import shutil
from google.colab import files

  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.1.3-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached pyarrow-18.0.0-cp313-cp313-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp313-cp313-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohttp-3.11.7-cp313-cp313-win_amd64.whl.metadata (8.0 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Checking for Rust toolchain....
      
      Cargo, the Rust package manager, is not installed or is not on PATH.
      This package requires Rust and Cargo to compile extensions. Install it through
      the system's package manager or via https://rustup.rs/
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


ModuleNotFoundError: No module named 'datasets'

In [ ]:

curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding
import pandas as pd
import os
import shutil
from google.colab import files

ModuleNotFoundError: No module named 'datasets'

## Import pre trained model (BERT)

In [5]:
# Pre trained model
model_path = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)
id2label = {0:"Real", 1:"Fake"}
label2id = {"Real":0, "Fake":1}

model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Freeze all layers first
for name, param in model.base_model.named_parameters():
  param.requires_grad = False

for name, param in model.base_model.named_parameters():
  if "pooler" in name:
    param.requires_grad = True

In [7]:
# Check layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

distilbert.embeddings.word_embeddings.weight False
distilbert.embeddings.position_embeddings.weight False
distilbert.embeddings.LayerNorm.weight False
distilbert.embeddings.LayerNorm.bias False
distilbert.transformer.layer.0.attention.q_lin.weight False
distilbert.transformer.layer.0.attention.q_lin.bias False
distilbert.transformer.layer.0.attention.k_lin.weight False
distilbert.transformer.layer.0.attention.k_lin.bias False
distilbert.transformer.layer.0.attention.v_lin.weight False
distilbert.transformer.layer.0.attention.v_lin.bias False
distilbert.transformer.layer.0.attention.out_lin.weight False
distilbert.transformer.layer.0.attention.out_lin.bias False
distilbert.transformer.layer.0.sa_layer_norm.weight False
distilbert.transformer.layer.0.sa_layer_norm.bias False
distilbert.transformer.layer.0.ffn.lin1.weight False
distilbert.transformer.layer.0.ffn.lin1.bias False
distilbert.transformer.layer.0.ffn.lin2.weight False
distilbert.transformer.layer.0.ffn.lin2.bias False
distilbe

## Import data

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mexwell/fake-reviews-dataset")

# Dataset path
print("Path to dataset files:", path)

# List files in the dataset directory
dataset_files = os.listdir(path)
print("Files in the dataset:", dataset_files)

df = pd.read_csv(os.path.join(path, dataset_files[0]))
df.head()

100%|██████████| 4.78M/4.78M [00:01<00:00, 3.71MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mexwell/fake-reviews-dataset/versions/1
Files in the dataset: ['fake reviews dataset.csv']


,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


# Pre-process data

In [9]:
df['label'] = df['label'].replace({'CG': 1, 'OR': 0})
df = df.drop(["category", "rating"], axis=1).rename(columns={'text_': 'Review'})
df.head()

<ipython-input-9-b42c252c93a8>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({'CG': 1, 'OR': 0})


,label,Review
0,1,"Love this! Well made, sturdy, and very comfor..."
1,1,"love it, a great upgrade from the original. I..."
2,1,This pillow saved my back. I love the look and...
3,1,"Missing information on how to use it, but it i..."
4,1,Very nice set. Good quality. We have had the s...


In [10]:
dataset = Dataset.from_pandas(df)

dataset = dataset.train_test_split(test_size=0.3)
train_dataset = dataset["train"]
temp_dataset = dataset["test"]
val_test_split = temp_dataset.train_test_split(test_size=0.5)
val_dataset = val_test_split["train"]
test_dataset = val_test_split["test"]

In [11]:
def preprocess(reviews):
  return tokenizer(reviews["Review"], truncation=True)

tokenized_train = train_dataset.map(preprocess, batched=True)
tokenized_val = val_dataset.map(preprocess, batched=True)
tokenized_test = test_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/28302 [00:00<?, ? examples/s]

Map:   0%|          | 0/6065 [00:00<?, ? examples/s]

Map:   0%|          | 0/6065 [00:00<?, ? examples/s]

# Create model

In [12]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Define metrics

In [13]:
# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)

    return {"Accuracy": acc, "AUC": auc}

## Training

In [19]:
# hyperparameters
lr = 2e-4
batch_size = 16
num_epochs = 5

In [16]:
training_args = TrainingArguments(
    output_dir="path",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,  # Increased batch size
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,  # Fewer epochs
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,  # Enable mixed precision
    gradient_accumulation_steps=2,  # Simulate larger batch size
    dataloader_pin_memory=True,
    report_to=None
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
os.environ["WANDB_MODE"] = "disabled"

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-17-cf4a9def8cc6>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
0,0.240800,0.269977,0.883000,0.971000
2,0.223200,0.199683,0.915000,0.976000
4,0.206900,0.210967,0.909000,0.976000


TrainOutput(global_step=4420, training_loss=0.22238673904902256, metrics={'train_runtime': 443.9041, 'train_samples_per_second': 318.785, 'train_steps_per_second': 9.957, 'total_flos': 1.0752191949177384e+16, 'train_loss': 0.22238673904902256, 'epoch': 4.997173544375354})

## Validate model

In [18]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_val)

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': 0.921, 'AUC': 0.978}


## Save model

In [20]:
print(os.getcwd())
os.mkdir()

/content


In [ ]:
import datetime
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_path = f"fine_tuned_model_lr{lr}_epochs{num_epochs}_{current_time}"
trainer.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [21]:


# Create directory and save the model and tokenizer
import datetime
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
save_dir = f"fine_tuned_DistilBERT_lr{lr}_epochs{num_epochs}_{current_time}"

os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Model and tokenizer saved in {save_dir}")



Model and tokenizer saved in fine_tuned_DistilBERT_lr0.0002_epochs5_2024-11-23_16-20-45


In [22]:
# Zip the model directory
zip_filename = "fine_tuned_DistilBERT_lr0.0002_epochs5_2024-11-23_16-20-45.zip"
shutil.make_archive("fine_tuned_DistilBERT_lr0.0002_epochs5_2024-11-23_16-20-45", 'zip', save_dir)
print(f"Zipped model saved as {zip_filename}")

# Download the zip file
files.download(zip_filename)

Zipped model saved as fine_tuned_DistilBERT_lr0.0002_epochs5_2024-11-23_16-20-45.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>